# Continuous Tomography

This Juypter Notebook is a tutorial for generating simulated data for the publication: ..................

Table of Contents


## 1. Manage Imports 
This section is to manage imports for libraries necessary to run the code. Data will be simulated from the Nanocage 3D volume.

In [1]:
import tomobase
import numpy as np
import tomobase.phantoms
import tomobase.processes
import tomobase.processes.alignments
import tomobase.tiltschemes


from skimage.metrics import peak_signal_noise_ratio, structural_similarity
vol = tomobase.phantoms.nanocage()


2024-12-01 22:00:34,893 - DEBUG - This is a debug message
ERROR:root:hyperspy module not found.


In [ ]:
def quantify_backlash(scheme, n_angles, cumulation):
    backlash = 2
    angles = np.array([scheme.get_angle() for i in range(n_angles)])
    added_backlash = 0 
    for i, angle in enumerate(angles):
        if i ==0:
            pass
        elif angle<angles[i-1]:
            angles[i] -= backlash
            added_backlash += backlash
    cumulative_backlash = cumulation + added_backlash
    index = n_angles
    
    return index, added_backlash, cumulative_backlash

def process_with_backlash(scheme, n_angles):
    backlash = 2
    angles = np.array([scheme.get_angle() for i in range(n_angles)])
    for i, angle in enumerate(angles):
        if i ==0:
            pass
        elif angle<angles[i-1]:
            angles[i] -= backlash
    ts = tomobase.processes.project(vol, angles)
    ts_noise = tomobase.processes.alignments.add_noise(ts, 0, 0, inplace=False)
    rec = tomobase.processes.reconstruct(ts_noise, method="sirt", iterations=100)
    ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
    psnr = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
    index = n_angles
    
    return index, ssim, psnr

In [ ]:
def process_calculate_psnr_ssim(scheme, n_angles):
        angles = np.array([scheme.get_angle() for j in range(n_angles)])
        ts = tomobase.processes.project(vol, angles)
        ts_noise = tomobase.processes.alignments.add_noise(ts, 0, 0, inplace=False)
        rec = tomobase.processes.reconstruct(ts_noise, method="sirt", iterations=100)
        ssim = structural_similarity(vol.data, rec.data, data_range=1.0)
        psnr = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)
        index = n_angles

        return index, psnr, ssim

## Processes
### Incremental Scanning

The following is a script for running a process for incremental scanning

In [ ]:
schemes = [tomobase.tiltschemes.Incremental(-64, 64, 32), 
           tomobase.tiltschemes.Incremental(-64, 64, 16),
           tomobase.tiltschemes.Incremental(-64, 64, 8),
           tomobase.tiltschemes.Incremental(-64, 64, 4),
           tomobase.tiltschemes.Incremental(-64, 64, 2)]

n_angles = [5, 9, 17, 33, 65]
array = np.zeros((len(schemes), 3))
for i, item in enumerate(schemes):
    array[i,0], array[i,1], array[i,2] = process_calculate_psnr_ssim(item, n_angles[i])
    
for i, item in enumerate(schemes):
    print(array[i,0], array[i,1], array[i,2])

### GRS

Loop for performing GRS acquisition and analysis

In [ ]:
num = 65
ssim = [None]*num
psnr = [None]*num
index = [None]*num

cumulation = 0
array = np.zeros((num, 3))
for i in range(num):
    scheme = tomobase.tiltschemes.GRS(-64, 64)
    angles = np.array([scheme.get_angle() for j in range(i+1)])
    array[i,0], array[i,1], array[i,2] = quantify_backlash(scheme, i+1, cumulation)
    cumulation = array[i,2]
        
for i in range(len(index)):    
    print(array[i,0], array[i,1], array[i,2])

### Binary

In [ ]:
num = 65
array = np.full((num, 3), np.nan)
for i in range(num):
    scheme = tomobase.tiltschemes.Binary(-64, 64, 8)
    angles = np.array([scheme.get_angle() for j in range(i+1)])

    full_set =False
    if i >= 2:
        if (angles[-1]>angles[-2]) and (angles[-2]<angles[-3]):
            full_set = True
        if (angles[-1]<angles[-2]) and (angles[-2]>angles[-3]):
            full_set = True
    
    if full_set:
        angles = np.array([scheme.get_angle() for j in range(i)])
        array[i,0], array[i,1], array[i,2] = process_with_backlash(scheme, i) 
        
    
array = array[~np.isnan(array).any(axis=1)]
for i in range(array.shape[0]):    
    print(array[i,0], array[i,1], array[i,2])

In [ ]:
scheme = tomobase.tiltschemes.Binary(-64, 64, 8)
angles = np.array([scheme.get_angle() for j in range(65)])
sino = tomobase.processes.project(vol, angles)

indices = np.where(np.diff(sino.angles) < 0)[0] + 1
sino.angles[indices] -= 0.5
rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
ssim_uncorrected = structural_similarity(vol.data, rec.data, data_range=1.0)
psnr_uncorrected = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)


indices = np.where(np.diff(sino.angles) < 0)[0] + 1
sino.angles[indices] -= 5
#sino, correction = tomobase.processes.alignments.backlash_correct(sino, extend_return=True)
#rec = tomobase.processes.reconstruct(sino, method="sirt", iterations=100)
#ssim_corrected = structural_similarity(vol.data, rec.data, data_range=1.0)
#psnr_corrected = peak_signal_noise_ratio(vol.data, rec.data, data_range =1.0)

print(correction, ssim_uncorrected, psnr_uncorrected, ssim_corrected, psnr_corrected)

2024-12-01 22:29:22,265 - INFO - type of volume: <class 'tomobase.data.volume.Volume'>
INFO:tomobase_logger:type of volume: <class 'tomobase.data.volume.Volume'>


4.874414982374111 0.8434934754075916 22.07997460708114 0.8505814880138284 22.241812592471568
